In [2]:
def matrix_chain_order(p):
    n = len(p) - 1  # number of matrices
    # m[i][j] stores the minimum number of multiplications for matrices i...j
    m = [[0] * n for _ in range(n)]
    # s[i][j] stores the index of the matrix at which to split
    s = [[0] * n for _ in range(n)]
    
    # Chain length L varies from 2 to n
    for L in range(2, n + 1):  # L is the chain length
        for i in range(n - L + 1):
            j = i + L - 1  # end index of the chain
            m[i][j] = float('inf')  # initialize with a large number
            for k in range(i, j):
                # Calculate cost of splitting at matrix k
                q = m[i][k] + m[k + 1][j] + p[i] * p[k + 1] * p[j + 1]
                if q < m[i][j]:
                    m[i][j] = q
                    s[i][j] = k  # record the split point
    
    # The minimum number of multiplications is in m[0][n-1]
    return m, s

def print_optimal_parenthesization(s, i, j):
    if i == j:
        return f"A{i+1}"
    else:
        return f"({print_optimal_parenthesization(s, i, s[i][j])} x {print_optimal_parenthesization(s, s[i][j] + 1, j)})"

def matrix_chain_multiplication(p):
    # Handle edge case where there's only one matrix
    if len(p) < 2:
        return "Invalid input. Need at least two matrices to multiply."
    
    m, s = matrix_chain_order(p)
    
    # Print the minimum number of operations
    min_operations = m[0][len(p) - 2]  # m[0][n-1] for n matrices
    print(f"Minimum number of scalar multiplications: {min_operations}")
    
    # Print the optimal parenthesization
    optimal_parenthesization = print_optimal_parenthesization(s, 0, len(p) - 2)
    print(f"Optimal Parenthesization: {optimal_parenthesization}")

# Example usage
if __name__== "_main_":
    # Input: matrix dimensions as an array, p = [p0, p1, ..., pn]
    # Matrix dimensions of the form A1: p0 x p1, A2: p1 x p2, ..., An: pn-1 x pn
    matrix_dimensions = [30, 35, 15, 5, 10, 20, 25]
    
    matrix_chain_multiplication(matrix_dimensions)

In [3]:
def MCO(dims):
    n = len(dims) - 1  #number of matrices
    if n < 1:
        raise ValueError("Matrix chain must have at least 2 matrices.")

    # Initialize m and s matrices
    m = [[0] * n for _ in range(n)]  
    s = [[0] * n for _ in range(n)]  

    # Loop over increasing lengths of the matrix chain
    for l in range(2, n + 1):  
        for i in range(n - l + 1):
            j = i + l - 1
            m[i][j] = float('inf')
            # Find the optimal place to split the matrix chain
            for k in range(i, j):
                q = m[i][k] + m[k + 1][j] + dims[i] * dims[k + 1] * dims[j + 1]
                if q < m[i][j]:
                    m[i][j] = q
                    s[i][j] = k

    return m, s

def optimal_order(s, i, j):
    # Build the optimal multiplication order
    if i == j:
        return f"A{i + 1}"
    else:
        left = optimal_order(s, i, s[i][j])
        right = optimal_order(s, s[i][j] + 1, j)
        return f"({left} x {right})"

def display(para, cost, order):
    print(f"Parameter: {para}")
    print(f"  Minimum Scalar Multiplications: {cost}")
    print(f"  Optimal Order: {order}")
    print()

def process_para(para):
    # Process each meteorological parameter
    for para, dims in para.items():
        # Perform matrix chain order calculation
        m, s = MCO(dims)
        order = optimal_order(s, 0, len(dims) - 2)
        cost = m[0][-1]
        display(para, cost, order)

# meteorological data
params = {
    "Temperature": [7, 3, 5, 4, 6, 8, 10, 5],  # 7 days, 3, 5, 4, 6, 8, 10, 5 measurements
    "Dew Point": [7, 2, 3, 4, 5, 6, 7, 5],   
    "Wind Speed": [7, 3, 4, 5, 6, 7, 8, 4],   
    "Cloud Cover": [7, 3, 6, 8, 3, 5, 7, 4]   
}

process_para(params)

Parameter: Temperature
  Minimum Scalar Multiplications: 771
  Optimal Order: (A1 x (((((A2 x A3) x A4) x A5) x A6) x A7))

Parameter: Dew Point
  Minimum Scalar Multiplications: 348
  Optimal Order: (A1 x (((((A2 x A3) x A4) x A5) x A6) x A7))

Parameter: Wind Speed
  Minimum Scalar Multiplications: 624
  Optimal Order: (A1 x (((((A2 x A3) x A4) x A5) x A6) x A7))

Parameter: Cloud Cover
  Minimum Scalar Multiplications: 507
  Optimal Order: (A1 x ((A2 x (A3 x A4)) x ((A5 x A6) x A7)))



In [12]:
def MCO(dims):
    if not dims:
        raise ValueError("Input dimension list cannot be empty.")
    if len(dims) < 2:
        raise ValueError("Matrix chain must have at least 2 matrices.")
    if any(d <= 0 for d in dims):
        raise ValueError("All dimensions must be positive integers.")
    if any(not isinstance(d, int) for d in dims):
        raise TypeError("All dimensions must be integers.")
    if any(d=="" for d in dims):
        raise TypeError("No strings allowed")

    n = len(dims) - 1  # number of matrices

    # Initialize m and s matrices
    m = [[0] * n for _ in range(n)]  
    s = [[0] * n for _ in range(n)]  

    # Loop over increasing lengths of the matrix chain
    for l in range(2, n + 1):  
        for i in range(n - l + 1):
            j = i + l - 1
            m[i][j] = float('inf')
            # Find the optimal place to split the matrix chain
            for k in range(i, j):
                try:
                    q = m[i][k] + m[k + 1][j] + dims[i] * dims[k + 1] * dims[j + 1]
                    if q < m[i][j]:
                        m[i][j] = q
                        s[i][j] = k
                except OverflowError:
                    raise OverflowError("Calculation resulted in a number too large to handle.")

    return m, s

def optimal_order(s, i, j):
    # Build the optimal multiplication order
    if i == j:
        return f"A{i + 1}"
    else:
        left = optimal_order(s, i, s[i][j])
        right = optimal_order(s, s[i][j] + 1, j)
        return f"({left} x {right})"

def display(para, cost, order):
    print(f"Parameter: {para}")
    print(f"  Minimum Scalar Multiplications: {cost}")
    print(f"  Optimal Order: {order}")
    print()

def process_para(para):
    # Process each meteorological parameter
    for para, dims in para.items():
        try:
            # Perform matrix chain order calculation
            m, s = MCO(dims)
            order = optimal_order(s, 0, len(dims) - 2)
            cost = m[0][-1]
            display(para, cost, order)
        except (ValueError, TypeError, OverflowError) as e:
            print(f"Error processing {para}: {e}")

# meteorological data
params = {
    "Temperature": [7, 3, 5, 4, 6, 8, 10, 5],  # 7 days, 3, 5, 4, 6, 8, 10, 5 measurements
    "Dew Point": [7, 2, 3, 4, 5, 6, 7, 5],   
    "Wind Speed": [7, 3, 4, 5, 6, 7, 8, 4],   
    "Cloud Cover": [7, 3, 6, 8, 3, 5, 7, 4],
    "Wind direction":[7, 4, 5, 1, 2, 8, 9, 5],
    "Invalid Input Example": [7, 0, 5, -3, 6],  # Contains zero and negative values
    "Single Matrix": [5],  # Only one matrix
    "Floating Point Test": [7, 3.5, 5, 4],  # Contains a float
    "Empty Test":[],#Empty input
    "Character Test":[7,"a",4,5] #Character in input
}

process_para(params)


Parameter: Temperature
  Minimum Scalar Multiplications: 771
  Optimal Order: (A1 x (((((A2 x A3) x A4) x A5) x A6) x A7))

Parameter: Dew Point
  Minimum Scalar Multiplications: 348
  Optimal Order: (A1 x (((((A2 x A3) x A4) x A5) x A6) x A7))

Parameter: Wind Speed
  Minimum Scalar Multiplications: 624
  Optimal Order: (A1 x (((((A2 x A3) x A4) x A5) x A6) x A7))

Parameter: Cloud Cover
  Minimum Scalar Multiplications: 507
  Optimal Order: (A1 x ((A2 x (A3 x A4)) x ((A5 x A6) x A7)))

Parameter: Wind direction
  Minimum Scalar Multiplications: 216
  Optimal Order: ((A1 x (A2 x A3)) x (((A4 x A5) x A6) x A7))

Error processing Invalid Input Example: All dimensions must be positive integers.
Error processing Single Matrix: Matrix chain must have at least 2 matrices.
Error processing Floating Point Test: All dimensions must be integers.
Error processing Empty Test: Input dimension list cannot be empty.
Error processing Character Test: '<=' not supported between instances of 'str' and '